# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(mongo.list_database_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", establishments.count_documents(query))
# Display the first document in the results using pprint
find = establishments.find(query)
pprint(find[0])

In [ ]:
# Convert the result to a Pandas DataFrame
find_df= pd.DataFrame(find)
# Display the number of rows in the DataFrame
print(len(find_df))
# # Display the first 10 rows of the DataFrame
find_df.head()

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName':{'$regex': 'London'}, 'RatingValue': {'$gte': '4'}}

# Use count_documents to display the number of documents in the result
print("number of documents:", establishments.count_documents(query))
# Display the first document in the results using pprint


In [ ]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(london)
# Display the number of rows in the DataFrame
print(len(london_df))
# Display the first 10 rows of the DataFrame
london_df.head

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
#finding latitude and longitude
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.

# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {'geocode.latitude':{'$lte':latitude+degree_search, '$gte':latitude-degree_search},
         'geocode.longitude':{'$lte':longitude+degree_search, '$gte':longitude-degree_search},'RatingValue':{'$eq':'5'}} 
sort =  [('scores.Hygiene',1)]

# Print the results
results = list(establishments.find(query).sort(sort).limit(5))

pprint(results)


In [ ]:
# Convert result to Pandas DataFrame
pf_df = pd.DataFrame(results)
pf_df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
hygiene_query = {'$match': {'scores.Hygiene':{'$eq': 0}}}
# 2. Groups the matches by Local Authority
local_query = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}
# 3. Sorts the matches from highest to lowest
hilo_query = sort_values = {'$sort': { 'count': -1 }}
# Print the number of documents in the result
query_pipeline = [hygiene_query, local_query, hilo_query]
list1 = list(establishments.aggregate(query_pipeline))
print(len(list1))
# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(list1)
# Display the number of rows in the DataFrame
print(len(hygiene_df))
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)